# Scaling Analysis

Author: Brain Gravelle (gravelle@cs.uoregon.edu)


All this is using the taucmdr python libraries from paratools
http://taucommander.paratools.com/


## Imports
This section imports necessary libraies, the metrics.py and utilities.py files and sets up the window.

In [1]:
# A couple of scripts to set the environent and import data from a .tau set of results
from utilities import *
from metrics import *
# Plotting, notebook settings:
%matplotlib inline  
#plt.rcParams.update({'font.size': 16})
import numbers
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.2e' % x)
pd.set_option('display.max_columns',100)
pd.set_option('max_colwidth', 70)

## Getting Data

TAU Commander uses TAU to run the application and measure it using runtime sampling techniques (similar to Intel VTune). Many customization options are available. For example, we may consider each function regardless of calling context, or we may decide to enable callpath profiling to see each context separately.

From the talapas_scaling application the following experiments are available. These use Talapas (with 28 thread Broadwell processors) and the build-ce (realistic) option for mkFit. The first six experiments use the --num-thr option to set the thread count which is intended to perform threading within the events. the last two add the --num-ev-thr option to set the event threads, so that all threads are used to process events in parallel and each event is processed by a single thread. 
* manual_scaling_Large_talapas		
* manual_scaling_Large_talapas_fullnode	
* manual_scaling_TTbar70_talapas		
* manual_scaling_TTbar70_talapas_fullnode
* manual_scaling_TTbar35_talapas
* manual_scaling_TTbar35_talapas_fullnode
* ev_thr_scaling_Large_talapas
* ev_thr_scaling_Large_talapas_fullnode

Additionally available in the cori_scaling application are the following. These were run on NERSC's Cori on the KNL with the default memory settings (quad - 1 NUMA domain, cache - MCDRAM as direct mapped cache). See http://www.nersc.gov/users/computational-systems/cori/running-jobs/advanced-running-jobs-options/ for more info on the KNL modes. Similar to the talapas scaling they use the build-ce option and threading within each event.
* manual_scaling_TTbar35
* manual_scaling_TTbar70
* manual_scaling_Large
* mixed_thr_scaling_Large


### Importing Scaling Data
Here we import the data. In this case we are using Cori data from the experiments with the threads working within each event using the TTbar35 file. Note that this box will take 10 or more minutes to run; please go enjoy a coffee while you wait.

In [2]:
# application = "talapas_scaling"
# experiment  = "manual_scaling_TTbar70_talapas"
# experiment  = "manual_scaling_Large_talapas"
# experiment = "ev_thr_scaling_Large_talapas"

application = "talapas_no_throttle_scaling"
experiment  = "manual_scaling_Large_talapas_fullnode"


# application = "cori_scaling"
# experiment  = "manual_scaling_TTbar35"
# experiment  = "manual_scaling_TTbar70"
# experiment  = "manual_scaling_Large"
# experiment  = "mixed_thr_scaling_Large"

path = ".tau/" + application + "/" + experiment + "/"
# note that this function takes a long time to run, so only rerun if you must

metric_data = get_pandas_scaling(path, callpaths=True)
print(metric_data.keys())
    

Found: 889 trials with 696 errors


[32, 48, 2, 3, 8, 16, 56]


In [12]:
if application == "talapas_scaling":
    metric_data = remove_erroneous_threads(metric_data,  [1, 2, 8, 16, 32, 48, 56])
elif application == "cori_scaling":
    metric_data = remove_erroneous_threads(metric_data,  [1, 4, 8, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256])

print metric_data.keys()
for kt in metric_data:
    print_available_metrics(metric_data[kt])
    print " "

[32, 48, 2, 3, 8, 16, 56]
PAPI_DP_OPS
PAPI_TLB_DM
DERIVED_L1_MISSRATE
PAPI_VEC_DP
PAPI_L3_TCM
PAPI_RES_STL
PAPI_SP_OPS
PAPI_LST_INS
PAPI_NATIVE_UOPS_RETIRED:STALL_CYCLES
PAPI_NATIVE_FP_ARITH:256B_PACKED_SINGLE
PAPI_VEC_SP
PAPI_NATIVE_FP_ARITH:256B_PACKED_DOUBLE
PAPI_L1_TCM
PAPI_BR_MSP
PAPI_TOT_CYC
 
PAPI_NATIVE_MACHINE_CLEARS:COUNT
PAPI_TLB_IM
PAPI_NATIVE_FP_ARITH:128B_PACKED_DOUBLE
PAPI_L3_TCM
PAPI_NATIVE_RESOURCE_STALLS:SB
 
PAPI_L2_TCA
PAPI_TLB_DM
PAPI_VEC_DP
PAPI_L2_TCM
DERIVED_CPI
PAPI_NATIVE_FP_ARITH:256B_PACKED_SINGLE
PAPI_L1_TCM
PAPI_BR_MSP
PAPI_NATIVE_MACHINE_CLEARS:COUNT
PAPI_DP_OPS
PAPI_RES_STL
PAPI_TOT_INS
PAPI_VEC_SP
PAPI_L3_TCA
PAPI_TLB_IM
PAPI_NATIVE_FP_ARITH:128B_PACKED_SINGLE
DERIVED_L1_MISSRATE
PAPI_L3_TCM
DERIVED_L3_MISSRATE
PAPI_NATIVE_FP_ARITH:SCALAR_DOUBLE
PAPI_NATIVE_RESOURCE_STALLS:SB
PAPI_NATIVE_FP_ARITH:SCALAR_SINGLE
PAPI_BR_INS
DERIVED_L2_MISSRATE
PAPI_NATIVE_FP_ARITH:128B_PACKED_DOUBLE
DERIVED_IPC
PAPI_LST_INS
PAPI_NATIVE_UOPS_RETIRED:STALL_CYCLES
PAPI_SP_OP

#### A list of metrics

In [13]:
print_available_metrics(metric_data[2])


PAPI_L2_TCA
PAPI_TLB_DM
PAPI_VEC_DP
PAPI_L2_TCM
DERIVED_CPI
PAPI_NATIVE_FP_ARITH:256B_PACKED_SINGLE
PAPI_L1_TCM
PAPI_BR_MSP
PAPI_NATIVE_MACHINE_CLEARS:COUNT
PAPI_DP_OPS
PAPI_RES_STL
PAPI_TOT_INS
PAPI_VEC_SP
PAPI_L3_TCA
PAPI_TLB_IM
PAPI_NATIVE_FP_ARITH:128B_PACKED_SINGLE
DERIVED_L1_MISSRATE
PAPI_L3_TCM
DERIVED_L3_MISSRATE
PAPI_NATIVE_FP_ARITH:SCALAR_DOUBLE
PAPI_NATIVE_RESOURCE_STALLS:SB
PAPI_NATIVE_FP_ARITH:SCALAR_SINGLE
PAPI_BR_INS
DERIVED_L2_MISSRATE
PAPI_NATIVE_FP_ARITH:128B_PACKED_DOUBLE
DERIVED_IPC
PAPI_LST_INS
PAPI_NATIVE_UOPS_RETIRED:STALL_CYCLES
PAPI_SP_OPS
PAPI_NATIVE_FP_ARITH:256B_PACKED_DOUBLE
PAPI_TOT_CYC


#### Metric metadata

In [5]:
print_metadata(metric_data[8])

TAU_MAX_THREADS                                    112
TAU_CUDA_BINARY_EXE                                None
TAU_MEASURE_TAU                                    off
Memory Size                                        131916412 kB
TAU_TRACK_SIGNALS                                  off
TAU_TRACK_IO_PARAMS                                off
CPU MHz                                            1200.062
Local Time                                         2018-04-30T22:08:35-07:00
CPU Type                                           Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz
TAU_OUTPUT_CUDA_CSV                                off
TAU_EBS_INCLUSIVE                                  0 usec
Node Name                                          n072
TAU_CALLPATH_DEPTH                                 100
CPU Cores                                          14
OS Machine                                         x86_64
TAU_SAMPLING                                       on
Cache Size                              

## Adding metrics

metrics are available in metrics.py. At this time the following can be added:
* add_IPC(metrics)          - Instructions per Cycle
* add_CPI(metrics)          - Cycles per instruction
* add_VIPC(metrics)         - vector instructions per cycle
* add_VIPI(metrics)         - vector instructions per instruction (i.e. fraction of total)
* add_L1_missrate(metrics)  - miss rate for L1 cache

for scaling data please use the add_metric_to_scaling_data(data, metric_func) function to add a metric

Here we add some predeefined metrics and print the top 10 functions with the best IPC

In [6]:
add_metric_to_scaling_data(metric_data, add_CPI)
add_metric_to_scaling_data(metric_data, add_IPC)
add_metric_to_scaling_data(metric_data, add_L1_missrate)
add_metric_to_scaling_data(metric_data, add_L2_missrate)
if application == 'cori_scaling': llc = True
else: llc = False
add_metric_to_scaling_data(metric_data, add_L3_missrate, llc)
print_available_metrics(metric_data, scaling=True)

metric_data[1]['DERIVED_IPC'].sort_values(by='Inclusive',ascending=False).head(10)

ERROR adding CPI to metric dictionary
ERROR adding CPI to metric dictionary
ERROR adding CPI to metric dictionary
ERROR adding CPI to metric dictionary
ERROR adding metric to thread count: 32
ERROR adding metric to thread count: 3
ERROR adding metric to thread count: 48
ERROR adding metric to thread count: 56
ERROR adding IPC to metric dictionary
ERROR adding IPC to metric dictionary
ERROR adding IPC to metric dictionary
ERROR adding IPC to metric dictionary
ERROR adding metric to thread count: 32
ERROR adding metric to thread count: 3
ERROR adding metric to thread count: 48
ERROR adding metric to thread count: 56
ERROR adding L1 MR to metric dictionary
ERROR adding L1 MR to metric dictionary
ERROR adding L1 MR to metric dictionary
ERROR adding L1 MR to metric dictionary
ERROR adding metric to thread count: 16
ERROR adding metric to thread count: 3
ERROR adding metric to thread count: 48
ERROR adding metric to thread count: 56
ERROR adding L2 MR to metric dictionary
ERROR adding L2 MR 

KeyError: 1

## Scaling Results

In this section we demo some scaling results with several different metrics.

We use the scaling plot function to plot the data vs thread count.
scaling_plot(data, inclusive=True, plot=True, function="\[SUMMARY\] .TAU application$", metric='PAPI_TOT_CYC', max=False)
* data = the full dictionary of scaling data 
* inclusive = determines if the inclusive data or exclusive data will be used
* plot = true makes a plot false does not
* function = the string that will be searched for to plot. Default looks at the whole application
* metric = the metric choosen from the above list
* max = use the max value or average value across the threads

### Scaling with total cycles vs the thread count
Here we plot the cycle count for each thread count as a proxy for execution time. We use the max cycle count rather than the average as this number will limit the time of execution.

In [ ]:
thread_list, tot_cyc_list = scaling_plot(metric_data, function="\[SUMMARY\] .TAU application$", max=True)

### Cycles per thread for each thread count
Here we show load balancing with a series of plots showing the cycle count per thread. We have one plot for each thread count used

In [ ]:
thread_cyc_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_TOT_CYC')

for kt in thread_list:
    print kt
    data = list(thread_cyc_data[kt])
    matplotlib.pyplot.bar(range(len(data)), data)
    matplotlib.pyplot.ylim(ymax=50000000000) 
    matplotlib.pyplot.show()

### L1 Missrate vs thread count
Similar to above these cells show the L1 missrates. In this case we want to get the plotting data for L1 acceses and misses but comupte the miss rate before plotting, so we set plot=False

In [ ]:
thread_list, L1A_data = scaling_plot(metric_data, plot=False, metric='PAPI_LST_INS')
thread_list, L1M_data = scaling_plot(metric_data, plot=False, metric='PAPI_L1_TCM')
    
L1_MR_list = [L1M_data[i] / L1A_data[i] for i in range(len(thread_list))]

plt = matplotlib.pyplot.plot(thread_list, L1_MR_list)

### L1 Miss rate by each thread of each thread count

In [ ]:
thread_L1A_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_LST_INS')
thread_L1M_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_L1_TCM')

MR_data = {}
for kt in thread_list:
#     print(thread_L1M_data[kt])
#     print(thread_L1A_data[kt])
    MR_data[kt] = thread_L1M_data[kt] / thread_L1A_data[kt]
    
for kt in thread_list:
    print kt
    data = list(MR_data[kt])
    matplotlib.pyplot.bar(range(len(data)), data)
    matplotlib.pyplot.ylim(ymax=0.05)
    matplotlib.pyplot.show()

### L1 Top 10 bad miss rates

In [ ]:
L1_data = select_metric_from_scaling(metric_data, 'DERIVED_L1_MISSRATE')
L1_MR_dict = {}
for n_thr in thread_list:
    L1_MR_dict[n_thr] = filter_libs_out(L1_data[n_thr]).sort_values(by='Inclusive',ascending=False)[["Inclusive"]]
print thread_list

In [ ]:
THREAD_COUNT = 8
# L1_MR_dict[THREAD_COUNT].head(10)

get_func_level_metric(L1_MR_dict[THREAD_COUNT], avg=False).head(10)

In [ ]:
THREAD_COUNT = 32
# L1_MR_dict[THREAD_COUNT].head(10)

get_func_level_metric(L1_MR_dict[THREAD_COUNT], avg=False).head(10)

In [ ]:
THREAD_COUNT = 80
# L1_MR_dict[THREAD_COUNT].head(10)

get_func_level_metric(L1_MR_dict[THREAD_COUNT], avg=False).head(10)

In [ ]:
THREAD_COUNT = 208
L1_MR_dict[THREAD_COUNT].head(10)
# TODO sum over all threads
# Average too

cyc_data = select_metric_from_scaling(metric_data, 'PAPI_TOT_CYC')

def get_func_level_metric(data, inclusive=True, avg=True, func = 'NULL'):
    '''
    data is a panda dataframe of one metric and one thread count
    returns a panda series of the metric averaged or summed over each thread
    note: for certain derived metrics (i.e. ratios) this won't work because it sums
    '''
    if inclusive: which='Inclusive'
    else: which='Exclusive'
        
    group_data = data.groupby(['region'])[[which]]

    if avg:
        metric_list = group_data.mean().sort_values(by=which,ascending=False)[[which]]
    else:
        metric_list = group_data.sum().sort_values(by=which,ascending=False)[[which]]

    if not func == 'NULL':
        metric_list = metric_list[metric_list.index.get_level_values('region').str.contains(func)][[which]]

    return metric_list

get_func_level_metric(cyc_data[8], avg=False, func='MkFinder::SelectHitIndices').head(10)

### L2 Missrate vs thread count
Similar to above these cells show the L2 missrates.

In [ ]:
thread_list, L2A_data = scaling_plot(metric_data, plot=False, metric='PAPI_L2_TCA')
thread_list, L2M_data = scaling_plot(metric_data, plot=False, metric='PAPI_L2_TCM')
    
L2_MR_list = [L2M_data[i] / L2A_data[i] for i in range(len(thread_list))]

plt = matplotlib.pyplot.plot(thread_list, L2_MR_list)

In [ ]:
thread_L2A_data = get_thread_level_metric_scaling(select_metric_from_scaling(metric_data, 'PAPI_L2_TCA'))
thread_L2M_data = get_thread_level_metric_scaling(select_metric_from_scaling(metric_data, 'PAPI_L2_TCM'))


L2_MR_data = {}
for kt in thread_list:
    L2_MR_data[kt] = thread_L2M_data[kt] / thread_L2A_data[kt]
    
for kt in thread_list:
    print kt
    data = list(L2_MR_data[kt])
    matplotlib.pyplot.bar(range(len(data)), data)
    matplotlib.pyplot.ylim(ymax=0.3)
    matplotlib.pyplot.show()

### L2 Top 10 bad miss rates

In [ ]:
L2_data = select_metric_from_scaling(metric_data, 'DERIVED_L2_MISSRATE')
L2_MR_dict = {}
for n_thr in thread_list:
    L2_MR_dict[n_thr] = filter_libs_out(L2_data[n_thr]).sort_values(by='Inclusive',ascending=False)[["Inclusive"]]
print thread_list

In [ ]:
THREAD_COUNT = 16
L2_MR_dict[THREAD_COUNT].head(10)

### L3 Missrate vs thread count
Similar to above these cells show the L3 missrates.

In [ ]:
if application == 'talapas_scaling':
    thread_list, LLA_data = scaling_plot(metric_data, plot=False, metric='PAPI_L3_TCA')
    thread_list, LLM_data = scaling_plot(metric_data, plot=False, metric='PAPI_L3_TCM')
else:
    thread_list, LLA_data = scaling_plot(metric_data, plot=False, metric='PAPI_NATIVE_LLC_REFERENCES')
    thread_list, LLM_data = scaling_plot(metric_data, plot=False, metric='PAPI_NATIVE_LLC_MISSES')
    
LL_MR_list = [LLM_data[i] / LLA_data[i] for i in range(len(thread_list))]

plt = matplotlib.pyplot.plot(thread_list, LL_MR_list)

In [ ]:
if application == 'talapas_scaling':
    thread_LLA_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_L3_TCA')
    thread_LLM_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_L3_TCM')
else:
    thread_LLA_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_NATIVE_LLC_REFERENCES')
    thread_LLM_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_NATIVE_LLC_MISSES')


LL_MR_data = {}
for kt in thread_list:
    LL_MR_data[kt] = thread_LLM_data[kt] / thread_LLA_data[kt]

def thread_bar_plots(data_dict, t_list, y=-1):
    for kt in t_list:
        print "Thread Count: %d" % kt
        data = list(data_dict[kt])
        matplotlib.pyplot.bar(range(len(data)), data)
        if y != -1:
            matplotlib.pyplot.ylim(ymax=y)
        matplotlib.pyplot.show()

thread_bar_plots(LL_MR_data, thread_list, 0.2)


### L3 Top 10 bad miss rates

In [ ]:
L3_data = select_metric_from_scaling(metric_data, 'DERIVED_L3_MISSRATE')
L3_MR_dict = {}
for n_thr in thread_list:
    L3_MR_dict[n_thr] = filter_libs_out(L3_data[n_thr]).sort_values(by='Inclusive',ascending=False)[["Inclusive"]]
print thread_list

In [ ]:
THREAD_COUNT = 32
L3_MR_dict[THREAD_COUNT].head(10)

In [ ]:
def scaling_plot_2(data, inclusive=True, plot=True, function="SelectHitInd", metric='DERIVED_L3_MISSRATE', max=False):
    '''
    data is the whole scaling data
    function is what to search in the call-path please use regular functions
        default looks at the whole application
    metric is the metric to plot

    returns lists of threads and metrics per thread (i.e. data to plot)
    '''
    if inclusive: which='Inclusive'
    else: which='Exclusive'

    metric_data = select_metric_from_scaling(data, metric)
    thread_list  = sorted(metric_data.keys())
    if max:
        data_list = [metric_data[kt][metric_data[kt].index.get_level_values('region').str.contains(function)][which].max() for kt in thread_list]
    else:
        # cause TAU has 2 of everything average is half
        data_list = [metric_data[kt][metric_data[kt].index.get_level_values('region').str.contains(function)][which].sum()/(2*kt) for kt in thread_list]
    
    if plot: plt = matplotlib.pyplot.plot(thread_list, data_list)

    return thread_list, data_list

t,d = scaling_plot_2(metric_data, inclusive=True, plot=True, function="MultHelixPropTransp", metric='DERIVED_L1_MISSRATE', max=False)
print d[8]
t,d = scaling_plot_2(metric_data, inclusive=True, plot=True, function="MultHelixPropTransp", metric='DERIVED_L1_MISSRATE', max=True)
print d[8]

# 0.0002159764720002364
# 0.024189364864026477

###  Resource Stalls vs thread count
Similar to above these cells show the Resource Stalls. In this case we have nothing to compute, so we simply call the function. Future work includes exploring the different types of stalls.

In [ ]:
thread_list, res_stall_data = scaling_plot(metric_data, metric='PAPI_RES_STL')

In [ ]:
thread_stall_data = get_thread_level_metric_scaling(metric_data, metric='PAPI_RES_STL')
thread_bar_plots(thread_stall_data, thread_list, 4000000000)

In [ ]:
alldata = combine_metrics(metric_data[32],'Inclusive')
cm = sns.light_palette("yellow", as_cmap=True)
correlations_pearson = alldata.corr('pearson').fillna(0)    # Other methods: 'kendall', 'spearman'
correlations_pearson.style.format("{:.2%}").background_gradient(cmap=cm)